# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****2f66
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain.agents import create_agent

In [3]:
import os
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("OLLAMA_MODEL") , temperature= 0 , base_url=os.getenv("OLLAMA_BASE_URL"))

In [6]:
from langsmith import uuid7
run_id = uuid7()  

In [7]:
agent = create_agent(
    model=model,
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [8]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why did the full‑stack developer go to the bar?

Because he heard the bartender was great at **“stacking”** drinks, but he kept getting a *404* when he tried to order a *404 Not Found* cocktail! 🍻


## values
You have seen this streaming mode in our examples so far. 

In [9]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why did the scarecrow win an award?  

Because he was outstanding in his field! 😄


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [10]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

**A Day in the Sun**

We packed a basket, bright and wide,  
With sandwiches, fruit, and a sweet surprise.  
The park was waiting, green and grand,  
With swings that squeaked and a gentle sand.

Mom hummed a tune while Dad set the shade,  
Kids chased butterflies, their laughter played.  
We shared a story, a joke, a grin,  
Under the sky, where clouds drifted in.

When the sun dipped low, we packed our cheer,  
Promising tomorrow’s adventure is near.  
Family’s love, simple and true,  
Turns ordinary moments into something new.

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [12]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='e803e652-c96b-4475-a357-f62006cf3747')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='e803e652-c96b-4475-a357-f62006cf3747'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-11-24T14:23:12.261431Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1172048792, 'load_duration': 104495833, 'prompt_eval_count': 128, 'prompt_eval_duration': 377853542, 'eval_count': 35, 'eval_duration': 678389291, 'logprobs': None, 'model_name': 'gpt-oss:20b', 'model_provider': 'ollama'}, id='lc_run--5e8cdb36-20ef-478a-98dd-af927403ee44-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'd8e71054-140f-445a-b9ec-c84d79920718', 'type': 'tool_call'}], usage_metadata={'input_tokens': 128, 'output_tokens': 35, 'total_tokens': 163}

In [13]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: SF')
('custom', 'Acquired data for city: SF')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [14]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: SF
Acquired data for city: SF
